In [3]:
import advent
advent.scrape(2024, 24)
data = advent.get_lines_doublenewline(24)
print(data[0])
print(data[1])

['x00: 1', 'x01: 0', 'x02: 1', 'x03: 1', 'x04: 0', 'x05: 0', 'x06: 1', 'x07: 1', 'x08: 0', 'x09: 1', 'x10: 1', 'x11: 1', 'x12: 1', 'x13: 0', 'x14: 1', 'x15: 0', 'x16: 0', 'x17: 0', 'x18: 0', 'x19: 0', 'x20: 1', 'x21: 1', 'x22: 0', 'x23: 1', 'x24: 0', 'x25: 0', 'x26: 1', 'x27: 1', 'x28: 1', 'x29: 1', 'x30: 1', 'x31: 1', 'x32: 1', 'x33: 1', 'x34: 0', 'x35: 1', 'x36: 1', 'x37: 1', 'x38: 0', 'x39: 1', 'x40: 1', 'x41: 0', 'x42: 0', 'x43: 1', 'x44: 1', 'y00: 1', 'y01: 0', 'y02: 0', 'y03: 1', 'y04: 1', 'y05: 0', 'y06: 0', 'y07: 0', 'y08: 0', 'y09: 0', 'y10: 0', 'y11: 1', 'y12: 0', 'y13: 1', 'y14: 0', 'y15: 0', 'y16: 0', 'y17: 1', 'y18: 1', 'y19: 1', 'y20: 1', 'y21: 0', 'y22: 0', 'y23: 0', 'y24: 1', 'y25: 1', 'y26: 0', 'y27: 0', 'y28: 1', 'y29: 1', 'y30: 1', 'y31: 1', 'y32: 1', 'y33: 1', 'y34: 1', 'y35: 1', 'y36: 1', 'y37: 0', 'y38: 1', 'y39: 1', 'y40: 0', 'y41: 0', 'y42: 0', 'y43: 1', 'y44: 1']
['ktr XOR cpc -> z27', 'hbk XOR fbg -> z13', 'rbm XOR tjp -> z36', 'y44 XOR x44 -> njr', 'x08 XOR y

In [21]:
registers = {}
for line in data[0]:
    r, v = line.split(': ')
    registers[r] = int(v)

lines_applied = 0
while lines_applied < len(data[1]):
    for line in data[1]:
        i1, op, i2, _, o = line.split()
        if i1 in registers and i2 in registers and o not in registers:
            lines_applied += 1
            if op == 'AND': registers[o] = registers[i1] & registers[i2]
            elif op == 'OR': registers[o] = registers[i1] | registers[i2]
            elif op == 'XOR': registers[o] = registers[i1] ^ registers[i2]
            else: raise ValueError()

biggest_z = int(sorted(registers.keys())[-1][1:])
print(biggest_z) # 45

result = [str(registers[f"z{i:02}"]) for i in range(biggest_z+1)]
print(int(''.join(result[::-1]), 2))

45
55920211035878


In [8]:
# Part 2

# Well, I decided to have some fun and read https://en.wikipedia.org/wiki/Adder_(electronics)
# The truth table for the full adder comes down to
# output = A xor B xor C_in (where A, B are inputs, C is carry from less significant bit)
# C_out =  (2 or more of A, B, C_in are 1)
#   this can be represented as '(a and b) or (b and c) or (a and c)', where a,b,c can be any
#  edit: after analyzing the input, they actually do something else, see lower cells

# Let's try analyzing the input a bit: this is probably not time efficient: I think the most
# time efficient would be to plot the input as a graph, and manually find the wrong edges

0


In [ ]:
# I used this online graphviz:
# https://dreampuf.github.io/GraphvizOnline/
# And pasted in the output of :
for line in data[1]:
    i1, op, i2, _, o = line.split()
    #print(f'{i1} -> {o} [label="{op}"];')
    #print(f'{i2} -> {o} [label="{op}"];')

# The 'fdp' engine is the most helpful

In [47]:
# Let's do some manual inspection
# What we see:
# x XOR y = s
# x AND y = b
# c_in XOR s = z (c_in from previous step)
# c_in AND s = a
# a OR b = c_out (carry for next step)

# We can see clear deviations from this in
# 17: 
# wvj XOR qwg -> cmv should go to z17
# tfc OR qhq -> z17 should go to cmv
# 30
# x30 and y30 -> z30 should go to rdg
# knj xor rvp -> rdg should go to z30

#  But unfortunately I couldn't easily find other mistakes
# Which was a bit dissapointing, I honestly thought this would easily let me find all 4 mistakes
# But at least it let me find the pattern

counter = 0
def fix_line(line):
    global counter
    if line == 'wvj XOR qwg -> cmv': return 'wvj XOR qwg -> z17'
    elif line == 'tfc OR qhq -> z17': return 'tfc OR qhq -> cmv'
    elif line == 'x30 AND y30 -> z30': return 'x30 and y30 -> rdg'
    elif line == 'knj XOR rvp -> rdg': return 'knj xor rvp -> z30'
    else:
        counter += 1
        return line
    
lines = [l.split() for l in data[1]]
def rules_from(s: str):
    return [l for l in lines if s == l[0] or s == l[2]]

def find_pattern(i: int = 1):
    xlines = rules_from(f"x{i:02}")
    if xlines[0][1] == 'XOR':
        s, b = xlines[0][4], xlines[1][4]
    else:
        b, s = xlines[0][4], xlines[1][4]
    # There should be two rules from s: both with c_out, one xor and one and
    slines = rules_from(s)
    assert len(slines) == 2, f"{i}, not 2 rules from s"
    cin = slines[0][0] if slines[0][2] == s else slines[0][2]
    cin2 = slines[1][0] if slines[1][2] == s else slines[1][2]
    assert cin == cin2, f"{i}, 2 rules from s have different partners"
    if slines[0][1] == 'XOR':
        z, a = slines[0][4], slines[1][4]
    else:
        a, z = slines[0][4], slines[1][4]
    assert z == f"z{i:02}", f"{i}, s xor c doesnt go to z"
    aline, bline = rules_from(a), rules_from(b)
    assert len(aline) == len(bline) == 1, f"{i}, a and b dont have 1 rule each"
    assert aline == bline, f"{i}, rule from a is not same as rule from b"

for i in range(1, 44):
    try:
        find_pattern(i)
    except AssertionError as e:
        print(e)
# That outputs 4 numbers as expected, we were missing 23 and 38

# 23:
# kkf AND pbw -> z23 should go to rmj
# kkf XOR pbw -> rmj should go to z23

# 38
# y38 AND x38 -> mwp should go to btb
# y38 XOR x38 -> btb should go to mwp

# This also explains why I couldnt find it by eye: these substitutions are such that the structure
# is exactly the same, the only thing that changes is replacing an AND with a XOR

17, s xor c doesnt go to z
23, s xor c doesnt go to z
30, s xor c doesnt go to z
38, not 2 rules from s


In [48]:
replaced_nodes = ['z17', 'rdg', 'cmv', 'z30', 'rmj', 'z23', 'btb', 'mwp']
print(','.join(sorted(replaced_nodes)))

btb,cmv,mwp,rdg,rmj,z17,z23,z30
